In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load 
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)
purchase_data.head()

Purchase ID             SN  Age Gender  Item ID  \
0            0        Lisim78   20   Male      108   
1            1    Lisovynya38   40   Male      143   
2            2     Ithergue48   24   Male       92   
3            3  Chamassasya86   24   Male      100   
4            4      Iskosia90   23   Male      131   

                                   Item Name  Price  
0  Extraction, Quickblade Of Trembling Hands   3.53  
1                          Frenzied Scimitar   1.56  
2                               Final Critic   4.88  
3                                Blindscythe   3.27  
4                                       Fury   1.44

In [2]:
purchase_data.shape

(780, 7)

In [3]:
purchase_data.dtypes

Purchase ID      int64
SN              object
Age              int64
Gender          object
Item ID          int64
Item Name       object
Price          float64
dtype: object

In [4]:
purchase_data.columns

Index(['Purchase ID', 'SN', 'Age', 'Gender', 'Item ID', 'Item Name', 'Price'], dtype='object')

## Player Count

In [5]:
# Collect length of unique player count and store in dictionary
total_players = len(purchase_data['SN'].value_counts())

d = {'Total Players': [total_players] }
total_players_df = pd.DataFrame(data=d)
total_players_df

Total Players
0            576

## Purchasing Analysis (Total)

In [6]:
# Collect length of unique item count, average price, purchase ID count, and total revenue, and store in dictionary
unique_items = len(purchase_data['Item ID'].value_counts())
average_price = purchase_data['Price'].mean()
total_purchases = purchase_data['Purchase ID'].count()
total_revenue = purchase_data['Price'].sum()

pat_df = {'Number of Unique Items' : [unique_items],
      'Average Price' : [average_price],
      'Number of Purchases' : [total_purchases],
      'Total Revenue' : [total_revenue]
     }
pat_df2 = pd.DataFrame(data=pat_df)

# Format columns into comma notation, dollar sign and two decimal places
pat_df2['Total Revenue'] = pat_df2['Total Revenue'].map("${:,.2f}".format)
pat_df2['Average Price'] = pat_df2['Average Price'].map("${:.2f}".format)
pat_df2

Number of Unique Items Average Price  Number of Purchases Total Revenue
0                     179         $3.05                  780     $2,379.77

## Gender Demographics

In [7]:
# Drop duplicate rows of same player
df_gd = purchase_data[['Gender','SN']].drop_duplicates(subset = 'SN')

# Acquire the counts into series
gender_count = df_gd['Gender'].value_counts(0)
gender_percent = df_gd['Gender'].value_counts(1)

# Convert the series into a single column data frame
gender_count_df = pd.DataFrame(gender_count)
gender_percent_df = round(pd.DataFrame(gender_percent) * 100, 2)

# Merge the data frames
gender_summary_df = gender_count_df.merge(gender_percent_df, left_index = True, right_index = True)
gender_summary_df.columns = ['Total Count', 'Percentage of Players']
gender_summary_df['Percentage of Players'] = gender_summary_df['Percentage of Players'].map("{:.2f}%".format)
gender_summary_df

Total Count Percentage of Players
Male                           484                84.03%
Female                          81                14.06%
Other / Non-Disclosed           11                 1.91%

## Purchasing Analysis (Gender)

In [8]:
purchase_data['Price'].dtype

dtype('float64')

In [9]:
# Group by gender
pag_df = purchase_data.groupby(['Gender'])

# Calculations
purchase_count = pag_df['Purchase ID'].count()
avg_purchase_price = round(pag_df['Price'].mean(), 2)
total_purchase_value = round(pag_df['Price'].sum(), 2)
purchase_value_per_gender = round(total_purchase_value / gender_count, 2)

# Create a dataframe
pag_df2 = pd.DataFrame([purchase_count, avg_purchase_price, total_purchase_value, purchase_value_per_gender])

# Transpose the data frame to set Gender as Index Column
pag_df2 = pag_df2.T

# Rename the columns accordingly
pag_df2.columns = ['Purchase Count', 'Average Purchase Price', 'Total Purchase Value', 'Avg Total Purchase per Person']
pag_df2['Average Purchase Price'] = pag_df2['Average Purchase Price'].map("${:.2f}".format)
pag_df2['Total Purchase Value'] = pag_df2['Total Purchase Value'].map("${:,.2f}".format)
pag_df2

Purchase Count Average Purchase Price  \
Female                          113.0                  $3.20   
Male                            652.0                  $3.02   
Other / Non-Disclosed            15.0                  $3.35   

                      Total Purchase Value  Avg Total Purchase per Person  
Female                             $361.94                           4.47  
Male                             $1,967.64                           4.07  
Other / Non-Disclosed               $50.19                           4.56

## Age Demographics

In [10]:
# Create bins for ages
bins = [0, 9, 14, 19, 24, 29, 34, 39, 150]
group_labels = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']

# Slice the data and place it into bins
pd.cut(purchase_data["Age"], bins, group_labels)

# Add new data into new column within orginal dataframe
purchase_data["Age Group"] = pd.cut(purchase_data["Age"], bins, labels=group_labels)

# Drop duplicate rows of same player and put into new dataframe
ad_df1 = purchase_data[['Gender','SN','Age Group']].drop_duplicates(subset = 'SN')

# Group by the Age Group into new dataframe
ad_df2 = ad_df1.groupby("Age Group").count()

# Acquire the age count from this data frame and calculate percentage
age_counts = ad_df2["Gender"]
age_percentages = round((age_counts / total_players)*100, 2)

# Insert into new data frame
ad_df3 = pd.DataFrame([age_counts, age_percentages])

# Transpose the data frame
ad_df3 = ad_df3.T
ad_df3.columns = ['Total Count', 'Percentage of Players']
ad_df3['Percentage of Players'] = ad_df3['Percentage of Players'].map("{:,.2f}%".format)
ad_df3

Total Count Percentage of Players
Age Group                                   
<10               17.0                 2.95%
10-14             22.0                 3.82%
15-19            107.0                18.58%
20-24            258.0                44.79%
25-29             77.0                13.37%
30-34             52.0                 9.03%
35-39             31.0                 5.38%
40+               12.0                 2.08%

## Purchasing Analysis (Age)

In [11]:
# Age Group column created earlier
paa_df1 = purchase_data[['Age Group','SN', 'Price']]

# Group by the Age Group into new dataframe
paa_df2 = paa_df1.groupby("Age Group")

# Obtain calculations
purchase_counts = paa_df2['SN'].count()
average_prices = round(paa_df2['Price'].mean(),2)
total_spent = round(paa_df2['Price'].sum(), 2)

# User previous age_counts variable, as it originates from data frame without duplicate players
spending_per_person = round(total_spent/age_counts,2)

# Create new data frame with new variables
paa_df3 = pd.DataFrame([purchase_counts, average_prices, total_spent, spending_per_person])
paa_df3 = paa_df3.T
paa_df3.columns = ['Purchase Count', 'Average Purchase Price', 'Total Purchase Value', 'Avg Total Purchase per Person']
paa_df3['Average Purchase Price'] = paa_df3['Average Purchase Price'].map("${:.2f}".format)
paa_df3['Total Purchase Value'] = paa_df3['Total Purchase Value'].map("${:,.2f}".format)
paa_df3['Avg Total Purchase per Person'] = paa_df3['Avg Total Purchase per Person'].map("${:.2f}".format)
paa_df3

Purchase Count Average Purchase Price Total Purchase Value  \
Age Group                                                               
<10                  23.0                  $3.35               $77.13   
10-14                28.0                  $2.96               $82.78   
15-19               136.0                  $3.04              $412.89   
20-24               365.0                  $3.05            $1,114.06   
25-29               101.0                  $2.90              $293.00   
30-34                73.0                  $2.93              $214.00   
35-39                41.0                  $3.60              $147.67   
40+                  13.0                  $2.94               $38.24   

          Avg Total Purchase per Person  
Age Group                                
<10                               $4.54  
10-14                             $3.76  
15-19                             $3.86  
20-24                             $4.32  
25-29                             $3.81  
30-34                             $4.12  
35-39                             $4.76  
40+                               $3.19

## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame

In [15]:
# Group by SN into new dataframe
ts_df1 = purchase_data.groupby('SN')

# Obtain calculations
purchase_counts = ts_df1['SN'].count()
average_spending = ts_df1['Price'].mean()
total_spending = ts_df1['Price'].sum()


# Create new data frame with new variables
ts_df2 = pd.DataFrame([purchase_counts, average_spending, total_spending])
ts_df2 = ts_df2.T
ts_df2.columns = ['Purchase Count', 'Average Purchase Price', 'Total Purchase Value']
ts_df2['Average Purchase Price'] = ts_df2['Average Purchase Price'].map("${:.2f}".format)
#ts_df2['Total Purchase Value'] = ts_df2['Total Purchase Value'].map("${:.2f}".format)

# Sort data
ts_df2.sort_values('Total Purchase Value', ascending=False)


Purchase Count Average Purchase Price  Total Purchase Value
SN                                                                      
Lisosia93               5.0                  $3.79                 18.96
Idastidru52             4.0                  $3.86                 15.45
Chamjask73              3.0                  $4.61                 13.83
Iral74                  4.0                  $3.40                 13.62
Iskadarya95             3.0                  $4.37                 13.10
...                     ...                    ...                   ...
Ililsasya43             1.0                  $1.02                  1.02
Irilis75                1.0                  $1.02                  1.02
Aidai61                 1.0                  $1.01                  1.01
Chanirra79              1.0                  $1.01                  1.01
Alo38                   1.0                  $1.00                  1.00

[576 rows x 3 columns]